In [325]:
import re
import nltk
import pandas as pd

from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from re import sub

In [326]:
def case_fold(word):
    lower_case = word.lower()
    return lower_case

In [327]:
def tagged_removal(text):
    text = sub(r"[\[].*?[\]]", " ", text)
    return text


In [328]:
def hashtag_removal(text):
    text = sub(r"#\S+", "",text)
    return text

In [329]:
#Normalize slangs and abbreviation

#load abbreviation dictionary
kamus = pd.read_csv("kamus_singkatan.csv")
replacers = {}
for x in range (0,kamus.shape[0]):
    abbreviation = kamus.at[x, 'singkatan']
    normalized = kamus.at[x,'normal']
    replacers = {**replacers, abbreviation:normalized}
    

# remove abbreviations function
def acronym_removal(text):
    for rk in replacers.keys():
        text = re.sub(r"(^|\s)("+ str(rk) + ")($|\s)", r"\1" + str(replacers[rk]) + r"\3", text)
    return text

In [330]:
#Stemming

factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stemming(text):
    stemmed = stemmer.stem(text)
    return stemmed

In [331]:
#read data
df = pd.read_csv("Twitter_Emotion_Dataset.csv")

text = df['tweet']

#collect preprocessing function
functions = [tagged_removal,
             hashtag_removal,
             case_fold,                         
             stemming,
             acronym_removal,
             stemming
             ]

#apply functions to the tweets
for function in functions:
    text = text.apply(function)

df['tweet'] = text

print(df)

      label                                              tweet
0         0  soal jalan jatibaru polisi tidak bisa gertak g...
1         0  sama cewek lho mirip harus bisa lebih rasain l...
2         1  kepingin gudeg mbarek bu hj amad foto dari goo...
3         0  jalan jatibaru bagi dari wilayah tanah abang a...
4         1  sharing alam saja kemarin jam 18 00 batalin ti...
...     ...                                                ...
4396      1  tahu kamu bahwa saat itu papa mejam mata dan t...
4397      0  sulit tetap calon wapresnya jokowi di pilpres ...
4398      0  5 masa depan tidak jelas lha iya gimana mau je...
4399      1  dulu benar ada mahasiswa teknik ui nembak paca...
4400      0  ya allah hanya engkau yang tahu rasa sakit di ...

[4401 rows x 2 columns]


In [332]:
df.to_csv('Cleaned_dataset.csv', columns=["tweet",'label'])